In [7]:
#imports
import pickle
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
import os
from matplotlib import pyplot as py
import numpy as np
path = "/home/lucky/Downloads/asm4/"
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from num2words import num2words
from collections import Counter
from collections import OrderedDict
from natsort import natsorted
from collections import Counter
import operator
import collections
import numpy as np

In [8]:
#listing all files
dir = [d[0] for d in natsorted(os.walk(path))]
files = []
btemp = False
for d in dir:
    if btemp == True:
        d = d+"/"
    if btemp == False:
        btemp = True
        continue
    for k in natsorted(os.listdir(d)): 
        files.append(d+k)

In [9]:
#preprocessing
def rem_stop(data):
    stop_words = stopwords.words('english') 
    for d in data:
        if d in stop_words:
            data.remove(d)
    return data

def tokenise(data):
    tokenizer = RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(data)

def stem(data):
    ps = PorterStemmer()
    for d in range(len(data)):
        data[d] = ps.stem(data[d])
    return data

def numTostr(data):
    for d in range(len(data)):
        try:
            data[d] = num2words(int(data[d]))
        except:
            pass
    return data
def to_lower(data):
    for d in range(len(data)):
        data[d] = data[d].lower()
    return data
def pprocess(data):
    data = tokenise(data)
    data = to_lower(data)
    data = rem_stop(data)
    data = numTostr(data)
    data = stem(data)
    return data

In [10]:
#reading files in list
fin_data = []
with open('findata.pickle', 'rb') as handle:
    fin_data = pickle.load(handle)
# for i in files:
#     op = open( i,'r',encoding="utf8", errors='ignore')
#     text = op.read().strip()
#     idx = text.find('\n\n')
#     text = text[idx+2:]
#     fin_data.append(pprocess(text))

In [11]:
# with open('findata.pickle', 'wb') as handle:
#     pickle.dump(fin_data, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [12]:
#calculatin doc frequency
df = {}
for i in range(len(fin_data)):
    tok = fin_data[i]
    for t in tok:
        if t not in df:
            df[t] = {i}
        else:
            df[t].add(i)
for i in df:
    df[i] = len(df[i])

In [13]:
#tf-idf functions
def caltf(countinDoc,docLength):
        return countinDoc/docLength

def calIdf(docfre,totaldoc):
        return np.log( (totaldoc+1)/(docfre+1) )

In [14]:
#storing tf-idf where key is doc,tok pair
tf_idf_doc = {}
d = 0
for i in range(len(fin_data)):
    doc_tok = fin_data[i]
    c = Counter(doc_tok)
    wc = len(doc_tok)
    for t in np.unique(doc_tok):
        termFre = caltf(c[t],wc)
        InDocFre =  calIdf(df.get(t),len(fin_data))
        tf_idf_doc[d,t] = termFre*InDocFre;
    d = d+1

In [2]:
#i could have saved this into pickel but loading from pickle is also taking same time
#thats why no need to save and upload 1.7 gb file 
#converting tf-idf into matrix
voc = list(df.keys())
def vectfidf():
    col = len(df)
    m = np.zeros((len(fin_data),col))
    for i in tf_idf_doc:
        try:
            colind = voc.index(i[1])
            rowind = i[0]
            m[rowind][colind] = tf_idf_doc[i]
        except:
            pass
    return m
mtf = vectfidf()

NameError: name 'df' is not defined

In [17]:
#calculate cosine
def calCos(vec1,vec2):
    num = np.dot(vec1,vec2)
    din = np.linalg.norm(vec1)*np.linalg.norm(vec2)
    sim = num/din
    return sim

In [18]:
#generate query vector
def gqV(q):
    v = np.zeros(len(voc))
    wc = Counter(q)
    totlen = len(q)
    for w in q:
        tf = caltf(wc[w],totlen)
        docfre = df[w]
        idf = calIdf(docfre,len(fin_data))
        idx = voc.index(w)
        v[idx] = tf*idf
    return v

In [19]:
#giving cosine similarity for top k docs
def cosSim(k,qv):
    cos = {}
    doc = 0
    for val in mtf:
        cos[doc] = calCos(val,qv)
        doc = doc+1
    res = sorted(cos.items(),key = lambda x:x[1],reverse=True)
    docList = []
    for i in res[:k]:
        docList.append((files[i[0]],i[0]))
    return docList

In [20]:
beta=0.75
gamma=0.25
#folder mappings
dir = {1:'comp.graphics',
       2:'rec.sport.hockey',
       3:'sci.med',
       4:'sci.space',
       5:'talk.politics.misc'}

In [21]:
#calculating centroid
def calcentr(dl):
    sum = np.zeros(len(voc))
    for d in dl:
        sum = np.add(d,sum)
    return np.divide(sum,len(dl))
#getting dir name of the file
def get_fold_name(name):
    trimmed = name[name.find('asm4')+5:]
    return trimmed[:trimmed.find('/')]

In [22]:
#plot pr curve
def plotpr(rl,pl):
    py.plot(rl, pl, marker='.', label='Logistic')
    py.xlabel('Recall')
    py.ylabel('Precision')
    py.show()
#plot tsne   
def plottsne(rdvl,nrdvl,qv):
    model = TSNE(n_components=2)
    transfrom = model.fit_transform(rdvl+nrdvl+[qv])
    rel, nonr, querv = np.array(transfrom[:len(rdvl)]), \
                       np.array(transfrom[len(rdvl):-1]), \
                       np.array(transfrom[-1])
    plt.scatter(rel[:,0],rel[:,1],color = 'blue')
    plt.scatter(nonr[:,0],nonr[:,1], color = 'red')
    plt.scatter(querv[0],querv[1], color = 'grey')
    plt.show()

In [23]:
oldrel=[]
#printing docs
def print_top_docs(list,crl):
    print("--------------------------------------------------")
    for i in list:
        if i in crl:
            print("*"+i[0])
        else: print(i[0])

In [24]:
#seprating docs into relevant and non relevant
def sepr_doc(fol,initList,p,qv):
    rdvl = []
    nrdvl = []
    nrd = 0
    pl = []
    rl = []
    crl = []
    rcount = 0
    apr = 0
    for i in initList:
        foldName = get_fold_name(i[0])
        nrd += 1
        if foldName == dir[int(fol)]:
            rcount+=1
            apr += rcount/nrd
        pl.append(rcount/nrd)
        rl.append(rcount/1000)
    c = 0
    for i in initList:
        if i not in oldrel:
            foldName = get_fold_name(i[0])
            if foldName == dir[int(fol)] and c<p:
                c+=1
                rdvl.append(mtf[i[1]])
                oldrel.append(i)
                crl.append(i)
            else:
                nrdvl.append(mtf[i[1]])
    print("Average precision is "+str(apr/100))
    plotpr(rl,pl)
    plottsne(rdvl,nrdvl,qv)
    return rdvl,nrdvl,crl

In [25]:
#main cell
#top docs
k = 100
#p% = 10
p = 10
q = input("Enter the query")
fol = input(" Input\n 1 for comp.graphics\n 2 for rec.sport.hockey\n"
      " 3 for sci.med\n 4 for sci.space\n 5 for talk.politics.misc ")
qv = gqV(pprocess(q))
i = 1
crl = []
while i<5:
    initList = cosSim(k,qv)
    print_top_docs(initList,crl)
    re,nr,crl = sepr_doc(fol,initList,p,qv)
    cr = calcentr(re)*beta
    cnr = calcentr(nr)*gamma
    qv = np.add(qv,np.subtract(cr,cnr))
    i+=1

/home/lucky/soft/anaconda3/envs/untitled4/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


Average precision is 0.22217150998286048
Average precision is 0.35343240610520615
Average precision is 0.5124255367528757
Average precision is 0.5475670550951321
